<H3>Capstone Project</H3><H4> Applied Data Science</H4>

<h2>Opening New Restaurant in Delhi India</h2>

Delhi is city, growing exponentially in Economical sector.With its population near about 1.9 crores it becomes one of the top market for investors.This notebook gives investors brief idea for opening new restaurants.So,let's get started.

<h3>1. Importing Necessary Packages and Libraries

In [5]:
import numpy as np 

import pandas as pd 
#pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


!pip install beautifulsoup4
from bs4 import BeautifulSoup # library to parse HTML and XML documents


import requests # library to handle requests



from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("All necessary Packeges Libraries imported.")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

All necessary Packeges Libraries imported.


<h3>2.Scrap data from Wikipedia page into a DataFrame

In [6]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store neighborhood data
neighborhoodList = []

In [9]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [10]:
# create a new DataFrame from the three lists
delhi_df = pd.DataFrame({"Neighborhood": neighborhoodList})
delhi_df.head()

,Neighborhood
0,Neighbourhoods of Delhi
1,Ashok Nagar (Delhi)
2,Ashok Vihar
3,Ashram Chowk
4,Babarpur


In [11]:
#Dropping first row of DataFrame
delhi_df=delhi_df.drop(delhi_df.index[0])

In [12]:
# get shape of the dataframe
delhi_df.shape

(136, 1)

<h3>3.Get the geographical coordinates

In [13]:
!pip install geocoder
import geocoder

In [15]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [16]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in delhi_df["Neighborhood"].tolist() ]

In [17]:
coords

[[28.692230000000052, 77.30127000000005],
 [28.690420000000074, 77.17606000000006],
 [28.710567996859677, 77.32694891535819],
 [28.507370000000037, 77.30347000000006],
 [28.507370000000037, 77.30347000000006],
 [28.652180044080556, 77.12977498073028],
 [28.800590000000057, 77.03473000000008],
 [28.549540000000036, 77.18170000000003],
 [28.69980000000004, 77.25917000000004],
 [28.595060000000046, 77.18573000000004],
 [28.656240000000025, 77.23233000000005],
 [28.538410000000056, 77.24831000000006],
 [28.676690000000065, 77.21762000000007],
 [28.633920000000046, 77.21978000000007],
 [28.607590000000073, 77.08721000000008],
 [28.654567834071344, 77.23341918654859],
 [28.62832000000003, 77.24727000000007],
 [28.605680000000064, 77.08222000000006],
 [28.560560000000066, 77.24679000000003],
 [28.573000000000036, 77.23360000000008],
 [28.591530000000034, 77.12942000000004],
 [28.594895372181515, 77.16726304796912],
 [28.684700000000078, 77.32775000000004],
 [28.679040000000043, 77.31476000000

In [18]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
delhi_df['Latitude'] = df_coords['Latitude']
delhi_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(delhi_df.shape)
delhi_df

(136, 3)


,Neighborhood,Latitude,Longitude
1,Ashok Nagar (Delhi),28.690420,77.176060
2,Ashok Vihar,28.710568,77.326949
3,Ashram Chowk,28.507370,77.303470
4,Babarpur,28.507370,77.303470
5,"Badarpur, Delhi",28.652180,77.129775
...,...,...,...
132,"Vasant Vihar, Delhi",28.646420,77.306150
133,Vivek Vihar subdivision,28.647800,77.164470
134,West Patel Nagar,28.621050,77.217100
135,Yamuna Pushta,28.700570,77.272100


In [19]:
# drop rows containing NaN values
delhi_df=delhi_df.dropna()

In [20]:
# check shape of DataFrame
delhi_df.shape

(135, 3)

In [21]:
# save the DataFrame as CSV file
delhi_df.to_csv("delhi_df.csv", index=False)

<H3>4.Create a map of Delhi with neighborhoods superimposed on top

In [22]:
# get the coordinates of Delhi
address = 'Delhi, India'

geolocator = Nominatim(user_agent="de_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India 28.6517178, 77.2219388.


In [23]:
# create map of Delhi using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(delhi_df['Latitude'], delhi_df['Longitude'], delhi_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_delhi)  
    
map_delhi

In [24]:
# save the map as HTML file
map_delhi.save('map_delhi.html')

<h3>5.Use the Foursquare API to explore the neighborhoods

In [28]:
CLIENT_ID = 'E1TUF5VL5TL0MV4DZF2FTXKCGPXPJ1TZGRLQ5KDCSV3K3VBH' # your Foursquare ID
CLIENT_SECRET = 'TRUEHTU4SETOWTR4M14YKHGNSNODZZR3XUIXWLBLVARPZVRU' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E1TUF5VL5TL0MV4DZF2FTXKCGPXPJ1TZGRLQ5KDCSV3K3VBH
CLIENT_SECRET:TRUEHTU4SETOWTR4M14YKHGNSNODZZR3XUIXWLBLVARPZVRU


<h5>Now, let's get the top 50 venues that are within a radius of 1000 meters.

In [29]:
radius = 1000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(delhi_df['Latitude'], delhi_df['Longitude'], delhi_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [30]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2509, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.69042,77.17606,Bellagio,28.696361,77.180021,Asian Restaurant
1,Ashok Nagar (Delhi),28.69042,77.17606,Domino's Pizza,28.693000,77.177000,Pizza Place
2,Ashok Nagar (Delhi),28.69042,77.17606,Rahul Egg Corner,28.688240,77.168599,Snack Place
3,Ashok Nagar (Delhi),28.69042,77.17606,Subway,28.696321,77.179983,Sandwich Place
4,Ashok Nagar (Delhi),28.69042,77.17606,Subway.,28.695571,77.171964,Sandwich Place


<h5>Let's check how many venues were returned for each neighborhood

In [31]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),10,10,10,10,10,10
Ashram Chowk,6,6,6,6,6,6
Babarpur,6,6,6,6,6,6
"Badarpur, Delhi",26,26,26,26,26,26
Bali Nagar,1,1,1,1,1,1
...,...,...,...,...,...,...
Vasant Kunj,41,41,41,41,41,41
"Vasant Vihar, Delhi",8,8,8,8,8,8
Vivek Vihar subdivision,10,10,10,10,10,10


<h5>Let's find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 194 uniques categories.


In [33]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Asian Restaurant', 'Pizza Place', 'Snack Place', 'Sandwich Place',
       'South Indian Restaurant', 'Indian Restaurant', 'Market',
       'Dessert Shop', 'Coffee Shop', 'Train Station',
       'Light Rail Station', 'American Restaurant', 'Hookah Bar',
       'Fast Food Restaurant', 'Café', 'Multiplex', 'Chinese Restaurant',
       'Ice Cream Shop', 'Shopping Mall', 'Electronics Store',
       'Clothing Store', 'Middle Eastern Restaurant', 'Music Venue',
       'Playground', 'Hotel', 'Grocery Store', 'Pharmacy',
       'Business Service', 'Karnataka Restaurant', 'Park',
       'Moroccan Restaurant', 'Pub', 'Nightclub', 'Historic Site',
       'Museum', 'Mosque', 'Monument / Landmark', 'Boutique',
       'Food & Drink Shop', 'Paper / Office Supplies Store',
       'Flea Market', 'Hardware Store', 'Restaurant',
       'Tibetan Restaurant', 'Italian Restaurant', 'Thai Restaurant',
       'Bakery', 'Mediterranean Restaurant', 'Convenience Store',
       'French Restaurant', 'BBQ Jo

<h3>6. Analyze Each Neighborhood

In [34]:
delhi_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

print(delhi_onehot.shape)
delhi_onehot.head()

(2509, 195)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,Antique Shop,Arcade,...,Tibetan Restaurant,Trail,Train Station,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Women's Store,Yoga Studio
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Let us see name of each VenueCategory
delhi_onehot.columns.tolist()

['Neighborhoods',
 'ATM',
 'Accessories Store',
 'Afghan Restaurant',
 'Airport',
 'Airport Food Court',
 'Airport Terminal',
 'American Restaurant',
 'Antique Shop',
 'Arcade',
 'Art Gallery',
 'Asian Restaurant',
 'Astrologer',
 'Athletics & Sports',
 'Australian Restaurant',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Basketball Court',
 'Bed & Breakfast',
 'Beer Garden',
 'Bengali Restaurant',
 'Big Box Store',
 'Bike Shop',
 'Bistro',
 'Bookstore',
 'Boutique',
 'Breakfast Spot',
 'Buffet',
 'Burger Joint',
 'Burrito Place',
 'Bus Station',
 'Business Service',
 'Café',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Cafeteria',
 'College Gym',
 'Comfort Food Restaurant',
 'Convenience Store',
 'Cosmetics Shop',
 'Cricket Ground',
 'Deli / Bodega',
 'Department Store',
 'Design Studio',
 'Dessert Shop',
 'Dim Sum Restaurant',
 'Diner',
 'Donut Shop',
 'Dumpling Restaurant',
 'Eastern European Re

<h5>Let us filter data where only 'Restaurant' as Venue Category is present

In [36]:
# filter neighborhood names that contain the word Restaurants
Reastaurant_names = list(venues_df.VenueCategory.unique())

VenueCategory_with_Restaurants = []

for x in Reastaurant_names:
    if "Restaurant" in x:
        VenueCategory_with_Restaurants.append(x)
        
VenueCategory_with_Restaurants

['Asian Restaurant',
 'South Indian Restaurant',
 'Indian Restaurant',
 'American Restaurant',
 'Fast Food Restaurant',
 'Chinese Restaurant',
 'Middle Eastern Restaurant',
 'Karnataka Restaurant',
 'Moroccan Restaurant',
 'Restaurant',
 'Tibetan Restaurant',
 'Italian Restaurant',
 'Thai Restaurant',
 'Mediterranean Restaurant',
 'French Restaurant',
 'Molecular Gastronomy Restaurant',
 'North Indian Restaurant',
 'Portuguese Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Udupi Restaurant',
 'Afghan Restaurant',
 'Vietnamese Restaurant',
 'Dumpling Restaurant',
 'Japanese Restaurant',
 'Mexican Restaurant',
 'Korean Restaurant',
 'Australian Restaurant',
 'Tapas Restaurant',
 'Mughlai Restaurant',
 'Modern European Restaurant',
 'English Restaurant',
 'Dim Sum Restaurant',
 'Scandinavian Restaurant',
 'Eastern European Restaurant',
 'Multicuisine Indian Restaurant',
 'Northeast Indian Restaurant',
 'Seafood Restaurant',
 'Indian Chinese Restaurant',
 'Bengali Restaurant',
 'Comfort 

In [37]:
# Taking sum of all columns containing Restaurant and populate the result in Restaurants column
delhi_onehot['Restaurants']=delhi_onehot.loc[:,VenueCategory_with_Restaurants].sum(axis=1)
delhi_onehot

,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,Antique Shop,Arcade,...,Trail,Train Station,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Women's Store,Yoga Studio,Restaurants
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,West Patel Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2505,West Patel Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2506,West Patel Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2507,West Patel Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


<h5>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
delhi_grouped = delhi_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(delhi_grouped.shape)
delhi_grouped

(133, 196)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,Antique Shop,Arcade,...,Trail,Train Station,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Women's Store,Yoga Studio,Restaurants
0,Ashok Nagar (Delhi),0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.300000
1,Ashram Chowk,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.333333
2,Babarpur,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.333333
3,"Badarpur, Delhi",0.0,0.0,0.0,0.0,0.0,0.000000,0.038462,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.307692
4,Bali Nagar,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,Vasant Kunj,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.268293
129,"Vasant Vihar, Delhi",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.250000
130,Vivek Vihar subdivision,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.100000
131,West Patel Nagar,0.0,0.0,0.0,0.0,0.0,0.021739,0.000000,0.0,0.0,...,0.0,0.021739,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.434783


<h5>Create a new DataFrame for Restaurants data only

In [39]:
delhi_restaurant = delhi_grouped[["Neighborhoods","Restaurants"]]
delhi_restaurant

,Neighborhoods,Restaurants
0,Ashok Nagar (Delhi),0.300000
1,Ashram Chowk,0.333333
2,Babarpur,0.333333
3,"Badarpur, Delhi",0.307692
4,Bali Nagar,0.000000
...,...,...
128,Vasant Kunj,0.268293
129,"Vasant Vihar, Delhi",0.250000
130,Vivek Vihar subdivision,0.100000
131,West Patel Nagar,0.434783


<h3>7. Cluster Neighborhoods

<h5>Run k-means to cluster the Delhi areas into 3 clusters.

In [40]:
# set number of clusters
kclusters = 3

delhi_grouped_clustering = delhi_restaurant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 2, 1, 0, 1, 0, 0, 0], dtype=int32)

In [41]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
delhi_merged = delhi_restaurant.copy()

# add clustering labels
delhi_merged["Cluster Labels"] = kmeans.labels_

delhi_merged.head()

,Neighborhoods,Restaurants,Cluster Labels
0,Ashok Nagar (Delhi),0.300000,2
1,Ashram Chowk,0.333333,0
2,Babarpur,0.333333,0
3,"Badarpur, Delhi",0.307692,2
4,Bali Nagar,0.000000,1


In [42]:
delhi_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
delhi_merged.head()

,Neighborhood,Restaurants,Cluster Labels
0,Ashok Nagar (Delhi),0.300000,2
1,Ashram Chowk,0.333333,0
2,Babarpur,0.333333,0
3,"Badarpur, Delhi",0.307692,2
4,Bali Nagar,0.000000,1


In [43]:
# merge delhi_merged with delhi_df to add latitude/longitude for each neighborhood
delhi_merged = delhi_merged.join(delhi_df.set_index("Neighborhood"), on="Neighborhood")

print(delhi_merged.shape)
delhi_merged.head()

(133, 5)


,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.300000,2,28.69042,77.176060
1,Ashram Chowk,0.333333,0,28.50737,77.303470
2,Babarpur,0.333333,0,28.50737,77.303470
3,"Badarpur, Delhi",0.307692,2,28.65218,77.129775
4,Bali Nagar,0.000000,1,28.80059,77.034730


In [44]:
# sort the results by Cluster Labels
print(delhi_merged.shape)
delhi_merged.sort_values(["Cluster Labels"], inplace=True)
delhi_merged

(133, 5)


,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
66,Meera Bagh,0.333333,0,28.520930,77.180110
99,Rajouri Garden,0.342857,0,28.565550,77.177220
96,Raisina Hill,0.324324,0,28.640658,77.185701
65,Mayur Vihar Phase - 3,0.333333,0,28.658410,77.087000
34,Gulmohar Park,0.320000,0,28.551090,77.204000
...,...,...,...,...,...
97,"Rajendra Nagar, Delhi",0.160000,2,28.645460,77.177760
98,Rajendra Place,0.275000,2,28.645620,77.122090
31,Greater Kailash,0.210526,2,28.696310,77.206450
102,"Rohini, Delhi",0.187500,2,28.683720,77.197460


<h4>Finally, let's visualize the resulting clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(delhi_merged['Latitude'], delhi_merged['Longitude'], delhi_merged['Neighborhood'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<h3>8.Examine Clusters

<h5>Cluster 1

In [47]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
66,Meera Bagh,0.333333,0,28.520930,77.180110
99,Rajouri Garden,0.342857,0,28.565550,77.177220
96,Raisina Hill,0.324324,0,28.640658,77.185701
65,Mayur Vihar Phase - 3,0.333333,0,28.658410,77.087000
34,Gulmohar Park,0.320000,0,28.551090,77.204000
94,Pitam Pura,0.357143,0,28.666330,77.125250
36,Inder Puri,0.333333,0,28.627940,77.090570
38,Jangpura,0.357143,0,28.546430,77.188960
91,Pandav Nagar,0.375000,0,28.669330,77.091730
41,Kailash Colony,0.333333,0,28.683770,77.201620


In [48]:
print(delhi_merged.loc[delhi_merged['Cluster Labels'] == 0].shape)

(49, 5)


<h5> Cluster 2

In [49]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
74,Nand Nagri,0.000000,1,28.678560,77.067640
77,Narela,0.111111,1,28.673690,77.283260
130,Vivek Vihar subdivision,0.100000,1,28.647800,77.164470
71,Munirka,0.000000,1,28.625080,76.997460
67,Mehrauli,0.076923,1,28.583630,77.164720
76,Naraina Area,0.000000,1,28.839770,77.076930
108,Saket District Centre,0.000000,1,28.584860,77.263560
106,Sagar Pur,0.000000,1,28.707970,77.049760
84,Nizamuddin West,0.000000,1,28.532490,77.278380
118,"Shanti Niketan, Delhi",0.000000,1,28.670060,77.251750


In [50]:
print(delhi_merged.loc[delhi_merged['Cluster Labels'] == 1].shape)

(29, 5)


<h5> Cluster 3

In [51]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
21,Dilshad Colony,0.200000,2,28.679040,77.314760
23,"Dwarka, Delhi",0.190476,2,28.648170,77.178320
68,Moti Bagh,0.176471,2,28.656720,77.129600
107,Saket (Delhi),0.200000,2,28.524090,77.206760
27,Geetanjali Enclave,0.250000,2,28.498280,77.136000
105,Safdarjung (Delhi),0.200000,2,28.605900,77.095540
28,Ghitorni,0.272727,2,28.634080,77.205760
109,Sarai Kale Khan,0.142857,2,28.550320,77.283430
25,"Gadaipur, Mehrauli, New Delhi",0.250000,2,28.660920,77.264310
117,Shankar Vihar,0.263158,2,28.577550,77.168130


In [52]:
print(delhi_merged.loc[delhi_merged['Cluster Labels'] == 2].shape)

(55, 5)


<h3> Observation

Examining Clusters we can see cluster 1 has moderate number of restaurants in neighborhoods,cluster 2 also has low to moderate number of restaurants while cluster 3 has large number restaurants.Hence to start a profitable business of opening restaurants,cluster 2 is best and competition is less here compared to other clusters.Cluster 1 is also good place to start business where competition is greater than cluster 2 and less than cluster 3. In brief most of the restaurants in Delhi are concentrated in Central area,where North-East part is best to set up restaurants for investors.